In [3]:
import joblib
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import configuration
from notebook_utils import notebookToPython
from label_reader import readLabels

In [4]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)

Write python file


In [5]:
trainingLabels = readLabels(job)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt...


In [6]:
X = []
y = []


fullDataPath = job.fullJoinFilePath(job.dataPath, job.trainingDataPath)

for filename, label in trainingLabels.items():
    audioSourceFilename = job.fullJoinFilePath(fullDataPath, filename + job.trainingDataExtension)
    
    audio, _ = librosa.load(audioSourceFilename, sr = job.sampleRate, duration = job.duration)

    mel_spectrogram = librosa.feature.melspectrogram(y = audio, sr = job.sampleRate, n_mels = job.numMels)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    if (mel_spectrogram.shape[1] < job.maxTimeSteps):
        padWidth = ((0, 0), (0, job.maxTimeSteps - mel_spectrogram.shape[1]))
        mel_spectrogram = np.pad(array=mel_spectrogram, pad_width=padWidth, mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :job.maxTimeSteps]

    X.append(mel_spectrogram)
    y.append(label)

In [7]:
X = np.array(X)
y = np.array(y)
y_encoded = to_categorical(y, job.numClasses)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)    # test data is 20% of all data

In [9]:
# Define CNN model architecture
input_shape = (job.numMels, X_train.shape[2], 1)  # Input shape for CNN (height, width, channels)
model_input = Input(shape=input_shape)


In [10]:
# TODO - why were these parameters selected? What purpose do they serve? Should they be configurable?
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(model_input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(0.5)(x)

model_output = Dense(job.numClasses, activation='softmax')(x)

In [11]:
model = Model(inputs=model_input, outputs=model_output)


In [12]:
model.compile(optimizer=job.optimizer, loss=job.loss, metrics=job.metrics)

In [13]:
# Train the Model
model.fit(X_train, y_train, batch_size=job.batchSize, epochs=job.numEpochs, validation_data=(X_test, y_test))

Epoch 1/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.8830 - loss: 3.3281 - val_accuracy: 0.8989 - val_loss: 0.2299
Epoch 2/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - accuracy: 0.9062 - loss: 0.2244 - val_accuracy: 0.9123 - val_loss: 0.2248
Epoch 3/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 39s 62ms/step - accuracy: 0.9218 - loss: 0.1828 - val_accuracy: 0.9466 - val_loss: 0.1366
Epoch 4/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - accuracy: 0.9393 - loss: 0.1339 - val_accuracy: 0.9458 - val_loss: 0.1267
Epoch 5/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - accuracy: 0.9524 - loss: 0.1168 - val_accuracy: 0.9624 - val_loss: 0.1039
Epoch 6/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.9639 - loss: 0.0915 - val_accuracy: 0.9736 - val_loss: 0.0800
Epoch 7/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.9727 - loss: 0.0734 - val_accuracy: 0.9730 - val_loss: 0.0799
Epoch 8/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 41s 65ms/step - accuracy: 0.9744 - loss: 0.0640 - 

In [14]:
joblib.dump(model, job.persistedModel)

['ASVspoof-2019-1.libjob']

### Test Model

In [17]:
import joblib
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import configuration
from notebook_utils import notebookToPython
from label_reader import readLabels

In [18]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)

Write python file


In [19]:
model = joblib.load(job.persistedModel)

In [20]:
predictions = model.predict(X_train)
predictions

635/635 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step


array([[4.2100716e-01, 5.7899290e-01],
       [9.3296356e-02, 9.0670365e-01],
       [1.0000000e+00, 1.6909349e-12],
       ...,
       [1.0000000e+00, 3.0720523e-21],
       [1.0000000e+00, 4.8419641e-22],
       [1.0000000e+00, 5.6659872e-09]], dtype=float32)

In [21]:
score = accuracy_score(y_train, predictions)
score

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets